In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import time
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
# Text preprocessing functions
def text_preprocessing(text):
    """Remove special characters, lowercase, and normalize spaces."""
    if pd.isna(text):
        return ""
    # Remove special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [8]:
platforms_meta_api = "https://api.rawg.io/api/platforms?key=f65a6008ebf44d2face1535b1518c4f3"
r_games = requests.get(platforms_meta_api)

json_platforms_data = r_games.json()

In [9]:
# Explore json file
for key,value in json_platforms_data.items():
    print(f"{key} : {value}")

count : 51
next : https://api.rawg.io/api/platforms?key=f65a6008ebf44d2face1535b1518c4f3&page=2
previous : None
results : [{'id': 4, 'name': 'PC', 'slug': 'pc', 'games_count': 547965, 'image_background': 'https://media.rawg.io/media/games/fc1/fc1307a2774506b5bd65d7e8424664a7.jpg', 'image': None, 'year_start': None, 'year_end': None, 'games': [{'id': 3498, 'slug': 'grand-theft-auto-v', 'name': 'Grand Theft Auto V', 'added': 21980}, {'id': 3328, 'slug': 'the-witcher-3-wild-hunt', 'name': 'The Witcher 3: Wild Hunt', 'added': 21620}, {'id': 4200, 'slug': 'portal-2', 'name': 'Portal 2', 'added': 20516}, {'id': 4291, 'slug': 'counter-strike-global-offensive', 'name': 'Counter-Strike: Global Offensive', 'added': 18240}, {'id': 5286, 'slug': 'tomb-raider', 'name': 'Tomb Raider (2013)', 'added': 17564}, {'id': 13536, 'slug': 'portal', 'name': 'Portal', 'added': 17484}]}, {'id': 187, 'name': 'PlayStation 5', 'slug': 'playstation5', 'games_count': 1244, 'image_background': 'https://media.rawg.io/

## Player Data

In [21]:
# load users data
users_df = pd.read_csv("../data/users_pref.csv")

# prin first few rows
users_df.head()

,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0
0,151603712,The Elder Scrolls V Skyrim,play,273.0,0
1,151603712,Fallout 4,purchase,1.0,0
2,151603712,Fallout 4,play,87.0,0
3,151603712,Spore,purchase,1.0,0
4,151603712,Spore,play,14.9,0


In [22]:
# rename columns
users_df.columns = ["user_id", "game_name", "behavior", "playtime", "unknown"]
# drop last column
users_df.drop(columns = ["unknown"], inplace = True)
# display quick info
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199999 entries, 0 to 199998
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    199999 non-null  int64  
 1   game_name  199999 non-null  object 
 2   behavior   199999 non-null  object 
 3   playtime   199999 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 6.1+ MB


In [24]:
# Clean game title
users_df["game_name_mod"] = users_df["game_name"].apply(text_preprocessing)
# remove games with 0 chars
users_df = users_df[users_df["game_name_mod"].str.len() > 0]
# convert Game Name Cleanded col to category
users_df["game_name_mod"] = users_df["game_name_mod"].astype("category")

# create new columns
users_df["purchase"] = users_df["behavior"].apply(lambda x: 1 if x == "purchase" or x == "play" else 0)
users_df["play"] = users_df["playtime"].apply(lambda x: 1 if x > 1 else 0)
users_df["hours_played"] = users_df["playtime"].apply(lambda x: 1 if x == 1 else x)

# display rows
users_df.head()

,user_id,game_name,behavior,playtime,game_name_mod,purchase,play,hours_played
0,151603712,The Elder Scrolls V Skyrim,play,273.0,the elder scrolls v skyrim,1,1,273.0
1,151603712,Fallout 4,purchase,1.0,fallout 4,1,0,1.0
2,151603712,Fallout 4,play,87.0,fallout 4,1,1,87.0
3,151603712,Spore,purchase,1.0,spore,1,0,1.0
4,151603712,Spore,play,14.9,spore,1,1,14.9


In [26]:
# drop columns
users_df.drop(columns = ["behavior", "playtime"], inplace = True)
users_df.head()

,user_id,game_name,game_name_mod,purchase,play,hours_played
0,151603712,The Elder Scrolls V Skyrim,the elder scrolls v skyrim,1,1,273.0
1,151603712,Fallout 4,fallout 4,1,0,1.0
2,151603712,Fallout 4,fallout 4,1,1,87.0
3,151603712,Spore,spore,1,0,1.0
4,151603712,Spore,spore,1,1,14.9


In [27]:
# Save cleaned dataframe for later use
users_df.to_csv("users_data.csv", index = False)

In [31]:
# unique games users had played
users_df["game_name_mod"].unique()

['the elder scrolls v skyrim', 'fallout 4', 'spore', 'fallout new vegas', 'left 4 dead 2', ..., 'warriors castles', 'romance of the three kingdoms maker', 'space colony', 'life is hard', 'executive assault']
Length: 5151
Categories (5151, object): ['007 legends', '0rbitalis', '1 2 3 kick it drop that beat like an ugly baby', '10 second ninja', ..., 'zotrix', 'zuma deluxe', 'zumas revenge', 'zumas revenge adventure']

In [34]:
# top 20 played/purchased games
users_df["game_name_mod"].value_counts()[:20]

game_name_mod
dota 2                            9682
team fortress 2                   4646
counterstrike global offensive    2789
unturned                          2632
left 4 dead 2                     1752
counterstrike source              1693
counterstrike                     1424
garrys mod                        1397
the elder scrolls v skyrim        1393
warframe                          1271
halflife 2 lost coast             1158
sid meiers civilization v         1150
robocraft                         1096
halflife 2 deathmatch             1021
portal                            1005
portal 2                           997
halflife 2                         995
heroes generals                    993
terraria                           956
counterstrike condition zero       904
Name: count, dtype: int64

## Games Data

In [11]:
# get all platforms
games_meta_api = "https://api.rawg.io/api/platforms?key=f65a6008ebf44d2face1535b1518c4f3"
all_platforms = [] # adding all games to this list

while games_meta_api:
    response = requests.get(games_meta_api)
    data = response.json()
    all_platforms.extend(data["results"])  # Add platforms to the list
    games_meta_api = data.get("next")

print(f"We have total {len(all_platforms)} platform")

We have total 51 platform


In [14]:
platforms_list = [{
    "id": p["id"],
    "name": p["name"],
    "slug": p["slug"],
    "Games Count": p["games_count"],
    "Image": p["image_background"],
    "year_start": p["year_start"],
    "year_end": p["year_end"]
} for p in all_platforms]

# convert to dataframe
platforms_df = pd.DataFrame(platforms_list)

platforms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           51 non-null     int64  
 1   name         51 non-null     object 
 2   slug         51 non-null     object 
 3   Games Count  51 non-null     int64  
 4   Image        51 non-null     object 
 5   year_start   2 non-null      float64
 6   year_end     0 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 2.9+ KB


In [15]:
top_platforms = platforms_df.sort_values(by="Games Count", ascending=False).head(5)
top_platforms

,id,name,slug,Games Count,Image,year_start,year_end
0,4,PC,pc,547965,https://media.rawg.io/media/games/fc1/fc1307a2...,NaN,None
50,171,Web,web,260078,https://media.rawg.io/media/screenshots/0e0/0e...,NaN,None
11,5,macOS,macos,106293,https://media.rawg.io/media/games/ec3/ec3a7db7...,NaN,None
12,6,Linux,linux,78896,https://media.rawg.io/media/games/f46/f466571d...,NaN,None
6,3,iOS,ios,77395,https://media.rawg.io/media/games/997/997ab4d6...,NaN,None


In [84]:
api_key = "f65a6008ebf44d2face1535b1518c4f3"
all_games = []

platform_ids = [4, 5, 6]  # fetch games from pc, macos and linux platforms

for platform_id in platform_ids:
    page = 1
    while True:
        params = {
            "key": api_key,
            "platforms": platform_id,
            "page_size": 40,
            "page": page
        }
        try:
            response = requests.get("https://api.rawg.io/api/games", params=params, timeout=5)
            response.raise_for_status()
            games = response.json().get("results", [])
            if not games:
                break

            for game in games:
                game_id = game["id"]

                # Fetch detailed info for each game
                detail_url = f"https://api.rawg.io/api/games/{game_id}"
                detail_params = {"key": api_key}
                try:
                    detail_resp = requests.get(detail_url, params=detail_params, timeout=10)
                    detail_resp.raise_for_status()
                    detail = detail_resp.json()

                    all_games.append({
                        "platform_id": platform_id,
                        "game_id": game_id,
                        "name": detail["name"],
                        "released": detail["released"],
                        "rating": detail["rating"],
                        "metacritic": detail.get("metacritic"),
                        "genres": ", ".join([g["name"] for g in detail.get("genres", [])]),
                        "tags": ", ".join([t["name"] for t in detail.get("tags", [])[:5]])
                    })

                    print(f"Fetched details for game ID: {game_id}")

                    time.sleep(1.5)  # Avoid rate limiting

                except requests.exceptions.RequestException as e:
                    print(f"Error fetching details for game ID {game_id}: {e}")
                    continue

            print(f"Platform {platform_id} - Page {page} done.")
            page += 1

        except requests.exceptions.RequestException as e:
            print(f"Error fetching game list for platform {platform_id} page {page}: {e}")
            break

games_df = pd.DataFrame(all_games)
print("Total detailed games fetched:", games_df.shape)

Fetched details for game ID: 3498
Fetched details for game ID: 3328
Fetched details for game ID: 4200
Fetched details for game ID: 4291
Fetched details for game ID: 5286
Fetched details for game ID: 13536
Fetched details for game ID: 12020
Fetched details for game ID: 5679
Fetched details for game ID: 28
Fetched details for game ID: 4062
Fetched details for game ID: 13537
Fetched details for game ID: 802
Fetched details for game ID: 3439
Fetched details for game ID: 4286
Fetched details for game ID: 32
Fetched details for game ID: 58175
Fetched details for game ID: 3070
Fetched details for game ID: 3939
Fetched details for game ID: 1030
Fetched details for game ID: 11859
Fetched details for game ID: 2454
Fetched details for game ID: 41494
Fetched details for game ID: 422
Fetched details for game ID: 10213
Fetched details for game ID: 766
Fetched details for game ID: 4459
Fetched details for game ID: 3272
Fetched details for game ID: 278
Fetched details for game ID: 29028
Fetched detail

In [90]:
# print first 5 rows
games_df.head()

,platform_id,game_id,name,released,rating,metacritic,genres,tags,name_mod
0,4,3498,Grand Theft Auto V,2013-09-17,4.47,92.0,Action,"Singleplayer, Steam Achievements, Multiplayer,...",grand theft auto v
1,4,3328,The Witcher 3: Wild Hunt,2015-05-18,4.65,92.0,"Action, RPG","Singleplayer, Full controller support, Atmosph...",the witcher 3 wild hunt
2,4,4200,Portal 2,2011-04-18,4.59,95.0,"Shooter, Puzzle","Singleplayer, Steam Achievements, Multiplayer,...",portal 2
3,4,4291,Counter-Strike: Global Offensive,2012-08-21,3.56,81.0,Shooter,"Steam Achievements, Multiplayer, Full controll...",counterstrike global offensive
4,4,5286,Tomb Raider (2013),2013-03-05,4.06,86.0,Action,"Singleplayer, Multiplayer, Full controller sup...",tomb raider 2013


In [92]:
# info about data
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3099 entries, 0 to 3098
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   platform_id  3099 non-null   int32         
 1   game_id      3099 non-null   int32         
 2   name         3099 non-null   object        
 3   released     3089 non-null   datetime64[ns]
 4   rating       3099 non-null   float64       
 5   metacritic   2118 non-null   float64       
 6   genres       3099 non-null   object        
 7   tags         3099 non-null   object        
 8   name_mod     3099 non-null   category      
dtypes: category(1), datetime64[ns](1), float64(2), int32(2), object(3)
memory usage: 264.4+ KB


In [93]:
# count of unique games
print(games_df["name"].value_counts())

# unquie games
print(games_df["name"].nunique())

name
Grand Theft Auto V                            1
NEOTOKYO                                      1
Volgarr the Viking                            1
Stubbs the Zombie in Rebel Without a Pulse    1
Nuclear Dawn                                  1
                                             ..
PC Building Simulator                         1
Wargroove                                     1
Total War: WARHAMMER                          1
LET IT DIE                                    1
Fun with Ragdolls: The Game                   1
Name: count, Length: 3099, dtype: int64
3099


In [94]:
# Clean game title
games_df["name_mod"] = games_df["name"].apply(text_preprocessing)
# remove games with 0 chars
games_df = games_df[games_df["name_mod"].str.len() > 0]
# convert Game Name Cleanded col to category
games_df["name_mod"] = games_df["name_mod"].astype("category")

# Last Played column as date
games_df["released"] = pd.to_datetime(games_df["released"])

# convert int64 columns into int32
int64_cols = games_df.select_dtypes(include='int64').columns
games_df[int64_cols] = games_df[int64_cols].astype('int32')

In [95]:
# save games_df optionally for later backup
games_df.to_csv("games.csv", index = False)

In [96]:
def clean_split_column(column):
    """
    Cleans and splits a column with comma-separated values into a list.
    Handles NaNs and strips whitespace.
    """
    return column.fillna('').apply(lambda x: [item.strip() for item in x.split(',') if item.strip()])

def encode_multilabel_column(df, column_name):
    """
    Applies MultiLabelBinarizer to a column of lists and returns the encoded DataFrame.
    """
    mlb = MultiLabelBinarizer()
    encoded = mlb.fit_transform(df[column_name])
    encoded_df = pd.DataFrame(encoded, columns=[f"{column_name}_{cls}" for cls in mlb.classes_], index=df.index)
    return encoded_df

In [103]:
games_df = pd.read_csv("games.csv")

In [104]:
# clean genres column
games_df['genres'] = clean_split_column(games_df['genres'])
# clean tags column
games_df['tags'] = clean_split_column(games_df['tags'])

In [105]:
# encode columns
genre_encoded = encode_multilabel_column(games_df, 'genres')
tag_encoded = encode_multilabel_column(games_df, 'tags')

# concatenate back to original DataFrame
clean_games = pd.concat([clean_games, genre_encoded, tag_encoded], axis=1)

# show columns
clean_games.columns

Index(['platform_id', 'game_id', 'name', 'released', 'rating', 'metacritic',
       'name_mod', 'genres_Action', 'genres_Adventure', 'genres_Arcade',
       ...
       'tags_street racing', 'tags_tracks', 'tags_true exclusive',
       'tags_unique', 'tags_vr mod', 'tags_w10 exclusive', 'tags_waves',
       'tags_wizard', 'tags_wizards', 'tags_work'],
      dtype='object', length=369)

In [107]:
# sort values by game_id and platform_id
clean_games = clean_games.sort_values(by = ["game_id", "platform_id"])
clean_games.head()

,platform_id,game_id,name,released,rating,metacritic,name_mod,genres_Action,genres_Adventure,genres_Arcade,...,tags_street racing,tags_tracks,tags_true exclusive,tags_unique,tags_vr mod,tags_w10 exclusive,tags_waves,tags_wizard,tags_wizards,tags_work
1309,4,21,DiRT 4,2017-06-06,3.61,78.0,dirt 4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
108,4,25,Middle-earth: Shadow of War,2017-09-27,3.85,82.0,middleearth shadow of war,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1494,4,26,Nex Machina,2017-06-19,3.90,89.0,nex machina,1,0,0,...,0,0,0,0,0,0,0,0,0,0
795,4,27,Pyre,2017-07-25,4.17,84.0,pyre,0,1,1,...,0,0,0,0,0,0,0,0,0,0
8,4,28,Red Dead Redemption 2,2018-10-26,4.59,96.0,red dead redemption 2,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
# Save preprocessed dataframe for later use
clean_games.to_csv("games_preprocessed.csv", index = False)

## Search Engine based on Text similarity

In [3]:
# Load games data if exist
games_df = pd.read_csv("../data/games_preprocessed.csv")

In [4]:
print(games_df.isnull().sum())
games_df = games_df.dropna(subset = ["name_mod"])
print(games_df.isnull().sum())

platform_id            0
game_id                0
name                   0
released              10
rating                 0
                      ..
tags_w10 exclusive     0
tags_waves             0
tags_wizard            0
tags_wizards           0
tags_work              0
Length: 369, dtype: int64
platform_id            0
game_id                0
name                   0
released              10
rating                 0
                      ..
tags_w10 exclusive     0
tags_waves             0
tags_wizard            0
tags_wizards           0
tags_work              0
Length: 369, dtype: int64


In [5]:
# instantiate object
vectorizer = TfidfVectorizer()
# apply it on games name
tfidf = vectorizer.fit_transform(games_df["name_mod"])

In [6]:
query = " grand    thieft auto   "
# preprocess query
cleaned_query = text_preprocessing(query)
print(cleaned_query)
# convert query to vector
query_vec = vectorizer.transform([cleaned_query])
# find similar items
similarities = cosine_similarity(query_vec, tfidf).flatten()
# print similarities
print(similarities)
# extract top 10 indices
indices = np.argpartition(similarities, -10)[-10:]
# final results similar to the query
results = games_df.iloc[indices]

grand thieft auto
[0. 0. 0. ... 0. 0. 0.]


In [7]:
results

,platform_id,game_id,name,released,rating,metacritic,name_mod,genres_Action,genres_Adventure,genres_Arcade,...,tags_street racing,tags_tracks,tags_true exclusive,tags_unique,tags_vr mod,tags_w10 exclusive,tags_waves,tags_wizard,tags_wizards,tags_work
1033,4,10575,The Shrouded Isle,2017-08-04,2.88,NaN,the shrouded isle,0,0,0,...,0,0,0,0,0,0,0,0,0,0
613,4,4414,Grand Theft Auto: Episodes from Liberty City,2009-10-29,4.11,NaN,grand theft auto episodes from liberty city,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3069,4,709004,Grand Theft Auto: San Andreas – The Definitive...,2021-11-11,3.06,NaN,grand theft auto san andreas the definitive ed...,1,1,0,...,0,0,0,0,0,0,0,0,0,0
619,4,4459,Grand Theft Auto IV,2008-04-29,4.26,95.0,grand theft auto iv,1,0,0,...,0,0,0,0,0,0,0,0,0,0
72,4,430,Grand Theft Auto: Vice City,2002-10-27,4.45,94.0,grand theft auto vice city,1,0,0,...,0,0,0,0,0,0,0,0,0,0
69,4,416,Grand Theft Auto: San Andreas,2004-10-26,4.51,93.0,grand theft auto san andreas,1,0,0,...,0,0,0,0,0,0,0,0,0,0
74,4,432,Grand Theft Auto III,2001-10-22,4.15,93.0,grand theft auto iii,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2451,4,52998,Grand Theft Auto,1997-10-21,3.86,NaN,grand theft auto,1,0,0,...,0,0,0,0,0,0,0,0,0,0
466,4,3498,Grand Theft Auto V,2013-09-17,4.47,92.0,grand theft auto v,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2450,4,52997,Grand Theft Auto 2,1999-10-22,4.01,70.0,grand theft auto 2,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
def fetch_query(query, vectorizer):
    cleaned_query = text_preprocessing(query)
    # convert query to vector
    query_vec = vectorizer.transform([cleaned_query])
    # find similar items
    similarities = cosine_similarity(query_vec, tfidf).flatten()
    # extract top 10 indices
    indices = np.argpartition(similarities, -5)[-5:]
    # final results similar to the query
    results = games_df.iloc[indices]
    # if there are duplicate rows
    # results = results.sort_values(by = "rating", ascending = False)
    # return final results
    return results

In [16]:
fetch_query("batman", vectorizer)

,platform_id,game_id,name,released,rating,metacritic,name_mod,genres_Action,genres_Adventure,genres_Arcade,...,tags_street racing,tags_tracks,tags_true exclusive,tags_unique,tags_vr mod,tags_w10 exclusive,tags_waves,tags_wizard,tags_wizards,tags_work
2729,4,295001,Batman: The Telltale Series,2016-08-02,3.75,79.0,batman the telltale series,0,1,0,...,0,0,0,0,0,0,0,0,0,0
273,4,2235,Batman: Arkham Origins,2013-10-25,3.95,74.0,batman arkham origins,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1569,4,16543,LEGO Batman,2008-09-23,3.77,80.0,lego batman,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2188,4,28623,Batman: Arkham City,2011-10-18,4.42,94.0,batman arkham city,1,0,0,...,0,0,0,0,0,0,0,0,0,0
416,4,3287,Batman: Arkham Knight,2015-06-23,4.24,81.0,batman arkham knight,1,0,0,...,0,0,0,0,0,0,0,0,0,0
